In [ ]:
! pip uninstall -y langchain
!pip install langchain langchain-openai langchain-pinecone

In [1]:
import os
OPENAI_API_KEY= "YOUR_API_KEY"
PINECONE_API_KEY= "YOUR_API_KEY"
os.environ['OPENAI_API_KEY']= OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Retrieval Augmented Generation avec Langchain 2 
### Retriver par utilisateurs

Lorsque vous créez une application utilisant du retrieval, vous devez souvent la concevoir en pensant à plusieurs utilisateurs. Cela signifie que vous pouvez stocker des données non pas pour un seul utilisateur, mais pour de nombreux utilisateurs différents, et qu'ils ne doivent pas être en mesure de voir les données de chacun d'entre eux. Cela signifie que vous devez configurer votre chaîne de récupération pour ne récupérer que certaines informations. Cela implique généralement 3 étapes:

##### 1. Assurez-vous que le système d'extraction que vous utilisez prend en charge plusieurs utilisateurs.

Pour l'instant, il n'y a pas d'indicateur ou de filtre unifié pour cela dans LangChain. Au contraire, chaque vectorstore et retriver peut avoir le sien, et peut être appelé différemment (namespaces, multi-tenancy, etc.). Pour les vectorstores, ceci est généralement exposé comme un argument de mot-clé qui est passé lors de `similarity_search`. En lisant la documentation ou le code source, déterminez si le retriever que vous utilisez prend en charge plusieurs utilisateurs et, si c'est le cas, comment l'utiliser.

>Note : ajouter de la documentation et/ou du support pour les utilisateurs multiples pour les extracteurs qui ne le supportent pas (ou ne le documentent pas) est une excellente façon de contribuer à LangChain.

##### 2. Ajouter ce paramètre en tant que champ configurable pour la chaîne

Cela vous permettra d'appeler facilement la chaîne et de configurer tous les drapeaux pertinents au moment de l'exécution. Voir [cette documentation](https://python.langchain.com/docs/expression_language/how_to/configure) pour plus d'informations sur la configuration.

##### 3. Appeler la chaîne avec ce champ configurable

Maintenant, au moment de l'exécution, vous pouvez appeler cette chaîne avec un champ configurable.

# Exemple

Voyons un exemple concret de ce que cela donne dans le code. Nous utiliserons Pinecone pour cet exemple.

> Il faudra créerun vectorstore sur l'interface web de pinecone au préalable avec le nom d'index de votre choix

Pour configurer Pinecone, définissez la variable d'environnement suivante :

PINECONE_API_KEY : votre clé d'API Pinecone

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name="test-example", embedding=embeddings)

vectorstore.add_texts(["j'ai travaillé chez meta"], namespace="patrick")
vectorstore.add_texts(["j'ai travaillé chez deepmind"], namespace="dimitri")

['d43f7b91-bcdd-4e5e-89e6-f16ab1ddcbc0']

Ceci n'obtiendra des documents que pour Patrick

In [18]:
vectorstore.as_retriever(search_kwargs={"namespace": "patrick"}).get_relevant_documents(
    "Où travaillait-t-il?"
)

[Document(page_content="j'ai travaillé chez meta")]

Ceci n'obtiendra des documents que pour Dimitri

In [17]:
vectorstore.as_retriever(search_kwargs={"namespace": "dimitri"}).get_relevant_documents(
    "Où travaillait-t-il?"
)

[Document(page_content="j'ai travaillé chez deepmind")]

Nous pouvons maintenant créer la chaîne que nous utiliserons pour répondre aux questions.

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Il s'agit d'une chaîne de questions-réponses de base.
template = """Réponds à la question en te basant uniquement sur le contexte suivant:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

Ici, nous indiquons que l'extracteur possède un champ configurable. Tous les récupérateurs de vectorstore ont un champ `search_kwargs`. Il s'agit simplement d'un dictionnaire, avec des champs spécifiques à vectorstore

In [20]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

Nous pouvons maintenant créer la chaîne à l'aide de notre retriever configurable.

In [21]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Nous pouvons maintenant invoquer la chaîne avec des options configurables. `search_kwargs` est l'identifiant du champ configurable. La valeur est le kwargs de recherche à utiliser pour Pinecone.

In [22]:
chain.invoke(
    "Où travaillait cet utilisateur?",
    config={"configurable": {"search_kwargs": {"namespace": "dimitri"}}},
)

'Cet utilisateur travaillait chez DeepMind.'

In [23]:
chain.invoke(
    "Où travaillait cet utilisateur?",
    config={"configurable": {"search_kwargs": {"namespace": "patrick"}}},
)

"L'utilisateur travaillait chez Meta."